# PYNQ DMA tutorial (Part 2: Using the DMA from PYNQ) 

---

## Aim 

* This notebook shows how to use the PYNQ DMA class to control an AXI DMA in a hardware design. This is the second part of a DMA tutorial. [PYNQ DMA tutorial (Part 1: Hardware design)](https://discuss.pynq.io/t/pynq-dma-tutorial-part-1-hardware-design/3133?u=cathalmccabe) shows how to build the Vivado hardware design used in this notebook. 


## References

* [PYNQ DMA tutorial (Part 1: Hardware design)](https://discuss.pynq.io/t/pynq-dma-tutorial-part-1-hardware-design/3133?u=cathalmccabe)
* [DMA Tutorial GitHub repository](https://github.com/cathalmccabe/pynq_tutorials/dma) with Tcl source to rebuild the hardware, and pre-compiled BIT and HWH
* [Xilinx PG021 AXI DMA product guide](https://www.xilinx.com/support/documentation/ip_documentation/axi_dma/v7_1/pg021_axi_dma.pdf) 


## Last revised
* 13 October 2021
   * Initial version


## Introduction

This overlay consists of an AXI DMA and an AXI Stream FIFO (input and output AXI stream interfaces). The FIFO connects the input and output streams of the DMA in a loopback configuration and will be used to explore the DMA and test the PYNQ DMA class.

![](./images/completed_design.png)

## Instantiate and download the overlay

In [7]:
from pynq import Overlay

ol = Overlay("/home/xilinx/bit/dma6_hp0.bit")

We can check the IPs in this overlay using the IP dictionary (*ip_dict*).

In [2]:
ol.ip_dict.keys()

dict_keys(['axi_dma_0', 'axi_gpio_0', 'PS/processing_system7_0'])

Check help for the DMA object

In [ ]:
ol.dma?

## Create DMA instances

Using the labels for the DMAs listed above, we can create two DMA objects.

In [8]:
dma = ol.dma
dma_send = dma.sendchannel
dma_recv = dma.recvchannel

## Read DMA
We will read some data from memory, and write to FIFO in the following cells.

The first step is to allocate the buffer. pynq.allocate will be used to allocate the buffer, and NumPy will be used to specify the type of the buffer. 

In [9]:
from pynq import allocate
import numpy as np

data_size = 100
input_buffer = allocate(shape=(data_size,), dtype=np.uint32)

The array can be used like any other NumPy array. We can write some test data to the array. Later the data will be transferred by the DMA to the FIFO. 

In [5]:
for i in range(data_size):
    input_buffer[i] = i + 0xcafe0000

Let's check the contents of the array. The data in the following cell will be sent from PS (DDR memory) to PL (streaming FIFO).

### Print first few values of buffer 

In [6]:
for i in range(10):
    print(hex(input_buffer[i]))

0xcafe0000
0xcafe0001
0xcafe0002
0xcafe0003
0xcafe0004
0xcafe0005
0xcafe0006
0xcafe0007
0xcafe0008
0xcafe0009


Now we are ready to carry out DMA transfer from a memory block in DDR to FIFO.

In [7]:
dma_send.transfer(input_buffer)

## Write DMA
Let's read the data back from FIFO stream, and write to MM memory. The steps are similar.

We will prepare an empty array before reading data back from FIFO.

### Print first few values of buffer 
(Check buffer is empty)

In [10]:
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)

for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00


In [13]:
dma_recv.transfer(output_buffer)

The next cell will print out the data received from PL (streaming FIFO) to PS (DDR memory). This should be the same as the data we sent previously.

### Print first few values of buffer 

In [14]:
for i in range(data_size):
    print('0x' + format(output_buffer[i], '02x'))

0xadcff103
0x1436587a
0x2020203
0xbee0f214
0x2547698b
0x2020204
0xcff10325
0x36587a9c
0x2020205
0xe0f21436
0x47698bad
0x2020206
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00


Verify that the arrays are equal

In [10]:
print("Arrays are equal: {}".format(np.array_equal(input_buffer, output_buffer)))

Arrays are equal: True


## Check DMA status, and trigger an error
Check the error and idle status

In [31]:
dma_recv.error?

Type:        property
String form: <property object at 0xac9a5028>
Docstring:   True if DMA engine is in an error state


In [198]:
dma_recv.error

False

In [ ]:
dma_recv.idle?

In [11]:
dma_recv.idle

False

First we will start a transfer, and check the DMA is not idle. We will then try to start another DMA transfer which shoudl trigger an error. 

In [55]:
dma_recv.transfer(output_buffer)

In [90]:
dma_recv.idle

False

Start another receive transfer while the DMA is not idle

In [37]:
dma_recv.transfer(output_buffer)

RuntimeError: DMA channel not idle

We can check the *running* state of the DMA

In [ ]:
dma_recv.running?

In [199]:
dma_recv.running

True

## Check the DMA register map
We can read back individual status bits as show above. It can be useful to read back the full register map which will give details on all control and status bits. The meaning of each register and each bit will not be covered. For more details you can refer to the product guide for the DMA.

In [102]:
dma.register_map

RegisterMap {
  MM2S_DMACR = Register(RS=0, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=0, IRQDelay=0),
  MM2S_DMASR = Register(Halted=0, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=0, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
  MM2S_CURDESC = Register(Current_Descriptor_Pointer=0),
  MM2S_CURDESC_MSB = Register(Current_Descriptor_Pointer=0),
  MM2S_TAILDESC = Register(Tail_Descriptor_Pointer=0),
  MM2S_TAILDESC_MSB = Register(Tail_Descriptor_Pointer=0),
  MM2S_SA = Register(Source_Address=0),
  MM2S_SA_MSB = Register(Source_Address=0),
  MM2S_LENGTH = Register(Length=0),
  SG_CTL = Register(SG_CACHE=0, SG_USER=0),
  S2MM_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  S2MM_DMASR = Register(Halted=0, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SG

As an example, we can compare the buffer (physical) addresses to the DMA source and destination addresses as shown in the register map. 

In [35]:
print("Input buffer address   :", hex(input_buffer.physical_address))
print("Output buffer address  :", hex(output_buffer.physical_address))
print("---")
print("DMA Source address     :", hex(dma.register_map.MM2S_SA.Source_Address))
print("DMA Destination address:", hex(dma.register_map.S2MM_DA.Destination_Address))

Input buffer address   : 0x1684a000
Output buffer address  : 0x16849000
---
DMA Source address     : 0x0
DMA Destination address: 0x16849000


## Free all the memory buffers
Don't forget to free the memory buffers to avoid memory leaks!

In [46]:
del input_buffer, output_buffer